In [1]:
import pandas as pd
import gensim as gs
import numpy as np
import sklearn
import rank_bm25
import string
from tqdm.autonotebook import tqdm
tqdm.pandas(desc="progress-bar")
from rank_bm25 import BM25Okapi
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import Stemmer

C:\Users\Stamina\AppData\Local\Temp\ipykernel_2820\2108125481.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stamina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_pickle("dataset_qdpairs.pkl")

In [3]:
df.head()

,query,document
id,,
386123400,DAVA: Davacı vekili dava dilekçesinde özetle; ...,DEĞERLENDİRME:Dava dilekçesi ve sair tüm evrak...
386123500,Davacı vekilinin mahkememize verdiği dava dile...,Delillerin Değerlendirilmesi :Tüm dosya kapsam...
386129400,Davacı şirket yetkilisi dava dilekçesinde özet...,Dosya kapsamında yer alan kayıt ve belgeler in...
386132200,Davacı vekili dava dilekçesinde özetle; Keşide...,DELİLLER VE GEREKÇE/Davacı vekilinin 19/01/201...
386172900,Davacı vekili dava dilekçesinde özetle; müvekk...,DELİLLER VE GEREKÇE / ... ATM'nin ... Esas sa...


In [4]:
def preprocess_str(str_to_process):
    # This is a turkish stemmer, doesn't work perfect but it is consistent at least
    stemmer = Stemmer.Stemmer('turkish')
    
    str_result = str_to_process
    # Remove non-chars
    str_result = gs.parsing.preprocessing.strip_multiple_whitespaces(gs.parsing.preprocessing.strip_numeric(gs.parsing.preprocessing.strip_non_alphanum(str_result)))
    # Lowercase str
    str_result = str_result.lower()
    # Remove stopwords
    str_result = gs.parsing.preprocessing.remove_stopwords(s=str_result, stopwords=stopwords.words("turkish"))
    # Split str
    str_result = str_result.split()
    # Stem words
    str_result = stemmer.stemWords(str_result)

    return str_result

def preprocess_df(df):
    # This is a turkish stemmer, doesn't work perfect but it is consistent at least
    stemmer = Stemmer.Stemmer('turkish')

    # Remove non-chars
    df = df.map(gs.parsing.preprocessing.strip_non_alphanum).map(
                                                gs.parsing.preprocessing.strip_numeric).map(gs.parsing.preprocessing.strip_multiple_whitespaces)
    # Lowercase str
    df = df.map(lambda s: s.lower())
    # Remove stopwords
    df = df.map(lambda s: gs.parsing.preprocessing.remove_stopwords(s=s, stopwords=stopwords.words("turkish")))
    # Split str
    df = df.map(lambda s: s.split())


    # Stem words - it takes long
    df = df.map(stemmer.stemWords)

    return df

In [7]:
import pickle

chunk_size = 20000
num_chunks = len(df) // chunk_size + 1 # should've been +1

for i in range(num_chunks):
    print(f"Chunk {i}")
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk = df.iloc[start_idx:end_idx]
    preprocessed_chunk = preprocess_df(chunk)
    preprocessed_chunk.to_pickle(f"chunks/preprocessed_chunk_{i}.pkl")
    del preprocessed_chunk

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12


In [4]:
import pandas as pd
import glob

# Get a list of all the pickles
pickle_files = glob.glob('chunks/preprocessed_chunk_*.pkl')
sorted_pickle_files = sorted(pickle_files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

# Read each pickle and concatenate them into a single dataframe
dfs = []
for pickle_file in sorted_pickle_files:
    print(pickle_file)
    df_chunk = pd.read_pickle(pickle_file)
    dfs.append(df_chunk)

df_merged = pd.concat(dfs)

# Write the merged dataframe back to a pickle file
df_merged.to_pickle('processed_dataset/dataset_qdpairs_processed.pkl')

Old/chunks\preprocessed_chunk_0.pkl
Old/chunks\preprocessed_chunk_1.pkl
Old/chunks\preprocessed_chunk_2.pkl
Old/chunks\preprocessed_chunk_3.pkl
Old/chunks\preprocessed_chunk_4.pkl
Old/chunks\preprocessed_chunk_5.pkl
Old/chunks\preprocessed_chunk_6.pkl
Old/chunks\preprocessed_chunk_7.pkl
Old/chunks\preprocessed_chunk_8.pkl
Old/chunks\preprocessed_chunk_9.pkl
Old/chunks\preprocessed_chunk_10.pkl
Old/chunks\preprocessed_chunk_11.pkl
Old/chunks\preprocessed_chunk_12.pkl
Old/chunks\preprocessed_chunk_13.pkl
